In [1]:
import numpy as np
import pandas as pd

import sys
import os
from tqdm import tqdm

sys.path.append("/kaggle")
from src.utils.post_process import post_process_for_seg
from src.utils.metrics import event_detection_ap

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
exp_id = "exp031"

sub_dfs = []
oof_dfs = []
for fold in tqdm(range(5)):
    keys = np.load(f"/kaggle/output/train/{exp_id}_fold{fold}/single/keys.npy")
    labels = np.load(f"/kaggle/output/train/{exp_id}_fold{fold}/single/labels.npy")
    preds = np.load(f"/kaggle/output/train/{exp_id}_fold{fold}/single/preds.npy")

    sub, oof = post_process_for_seg(
        keys=keys,
        preds=preds[:, :, [1, 2]],
        score_th=0.02,
        distance=10,
        penguin_pp=True,
    )
    sub_dfs.append(sub)
    oof_dfs += oof

100%|██████████| 5/5 [02:04<00:00, 24.86s/it]


In [3]:
sub = pd.concat(sub_dfs).reset_index(drop=True)
labels = pd.read_csv("/kaggle/data/train_events.csv").dropna()
score = event_detection_ap(labels, sub)
score

0.7898762399523673

In [4]:
oof = pd.concat(oof_dfs).reset_index(drop=True)
oof[["wakeup_oof", "onset_oof"]] = oof[["wakeup_oof", "onset_oof"]].astype(float)
oof.to_parquet(f"/kaggle/output/train/{exp_id}_oof.parquet", index=False)